## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content, Personalization
import asyncio
import openai
import pandas as pd


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
load_dotenv(override=True)

True

## Step 1: Agent workflow

In [4]:
file_path = r"c:\Users\ninic\projects\agents\2_openai\community_contributions\friends_data.csv"
df = pd.read_csv(file_path)

print(df.head())  # Optional: Display first few rows to confirm successful reading


tokens = 600


                      EMAIL FIRST_NAME  LAST_NAME  GENDER ADDRESS_LINE_2
0          drey@proximus.be  Dominique  Reyntjens  female        Belgium
1  vladimirmylle@hotmail.be   Vladimir      Mylle    male        Belgium
2   laura.mylle@hotmail.com      Laura   Myllette  female        Belgium
3        ninicoe0@gmail.com       Nini     Coenen  female        Belgium


In [5]:

# Load recipients' emails from CSV file

recipients = df.to_dict(orient="records")  # Convert to list of dictionaries
email_sender = "ninicoe0@gmail.com"

In [6]:
instruction_orchestrator ="you are the orchestrator of a story of 4 friends of which one has the idea to organise a trip and send out an email to the three other friends. You will orchestrate the flow of agents: one will generate the story of the 4 friends, another agent will generate on the gender and personality of the friends the subject and body text of the emails. After the last phase ( assessment of the email) of the flow, you will handover the subject and the email body to the email_sender agent who will send out the emails."
instructions_storyteller = f"Generate a compelling short story (max {tokens} tokens) about four friends, information on name and gender to be found in {df}, ensuring the story naturally leads to one of them organizing a holiday trip. The story should reflect the personalities, interests, and relationships of the friends to provide useful context for an email-generating agent. The tone should be engaging, slightly conversational, and realistic, balancing storytelling with practical details to support email personalization. The story should conclude with a natural setup for an email being written." 
instructions_male_emailagent =f"Write a direct and energetic email tailored to male friends based on the story context. See that you reflect the personality of the recipient in the tone of the email. Information on the gender of the friends can be foud in {df}"
instructions_female_emailagent = f"Write a warm and engaging email tailored to female friends based on the story context.See that you reflect the personality of the recipient in the tone of the email. Information on the gender of the friends can be foud in {df}"
instructions_evaluator = "Assess all the emails for clarity and relevance. Give an assessment per email that has been created"

In [7]:
storyteller_userprompt = "write a short story"

In [8]:
instructions_html_converter = """You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.
Convert the emails to HTML as follows: male_email to HTML_email_male and female_email to HTML_email_female.
"""

instructions_email_sender = "You are an email formatter and sender. You receive the subject and body of an email to be sent. \
You first use the the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

In [9]:
html_converter = Agent(name="HTML email body converter", instructions=instructions_html_converter, model="gpt-4o")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body, male_email to HTML_email_male and female_email to HTML_email_female")

In [10]:

async def orchestrate_workflow():
    message = "you will use this tool to start the workflow and create a vivid subject and body/text for the email"
    # Step 1: Generate Story
    storyteller_agent = Agent(
        name="StoryTeller",
        instructions=instructions_storyteller
    )
    story_result = await Runner.run(storyteller_agent, storyteller_userprompt)
  
    # Step 2: Generate email_prompt
    email_prompt = f"Based on this story: {story_result.final_output}, generate an email inviting the friends to the trip."

    # Step 3: Run Male & Female Email Agents in Parallel
    male_email_agent = Agent(
        name="MaleEmailAgent",
        instructions= instructions_male_emailagent
    )
    female_email_agent = Agent(
        name="FemaleEmailAgent",
        instructions=instructions_female_emailagent
    )

    male_email_task = Runner.run(male_email_agent, email_prompt)
    female_email_task = Runner.run(female_email_agent, email_prompt)

    male_email_result, female_email_result = await asyncio.gather(male_email_task, female_email_task)

    # Step 4: Evaluator Reviews Both Emails
    evaluator_agent = Agent(
        name="Evaluator",
        instructions= instructions_evaluator
    )

    eval_male_task = Runner.run(evaluator_agent, male_email_result.final_output)
    eval_female_task = Runner.run(evaluator_agent, female_email_result.final_output)

    eval_male_result, eval_female_result = await asyncio.gather(eval_male_task, eval_female_task)
# return in dictionnary as the output of the workflow was to hugh
    return {
    "story": story_result.final_output,
    "male_email": male_email_result.final_output,
    "female_email": female_email_result.final_output,
    "eval_male": eval_male_result.final_output,
    "eval_female": eval_female_result.final_output
} 



 # Check how many values are returned # Checprint(len(result)) k how many values are returned

# Run the orchestrated workflow
result = await orchestrate_workflow()
print(len(result)) # Check how many values are returned # Checprint(len(result)) k how many values are returned
# Access values from the dictionary
print("\nGenerated Story:\n", result["story"])
print("\nEmail for Male Friends:\n", result["male_email"])
print("\nEvaluation for Male Email:\n", result["eval_male"])
print("\nEmail for Female Friends:\n", result["female_email"])
print("\nEvaluation for Female Email:\n", result["eval_female"])

5

Generated Story:
 In the heart of Belgium, four friends sat around a cozy café table, surrounded by the aroma of fresh pastries and the rhythmic hum of early morning chatter. Dominique Reyntjens, an adventurous spirit with an eye for detail, leafed through a travel magazine, absentmindedly sipping her espresso. Beside her, Vladimir Mylle, a history enthusiast with a penchant for old films, recounted a story that had them all chuckling.

Vladimir’s sister, Laura Myllette, artistic and thoughtful, had brought along her sketchbook. She was capturing the café's essence with quick, confident strokes. Across the table, Nini Coenen, an energetic and practical problem-solver, was already on her phone, researching travel spots that Dominique had been considering.

“Dom, this place in Greece looks perfect for your hiking obsession,” Nini chimed in, pushing her phone toward Dominique.

Dominique’s eyes lit up as she scanned the screen. “It does look amazing. A mix of trails and beaches,” she m

In [11]:
html_converter_agent = Agent(
    name="HTMLConverter",
    instructions=instructions_html_converter,
    
)
message_male_email = f"convert this email to HTML: {result['male_email']}"
message_female_email = f"convert this email to HTML: {result['female_email']}"

with trace("Automated emailconverter"):
# Convert emails into HTML format
    html_email_task_male = Runner.run(html_converter_agent, message_male_email)
    html_email_task_female = Runner.run(html_converter_agent, message_female_email)

html_email_male, html_email_female = await asyncio.gather(html_email_task_male, html_email_task_female)


 

In [12]:

sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))


In [13]:

@function_tool
def send_email(subject: str, html_content: str, recipient_email: str) -> int:
    """
    Send emails to all friends using SendGrid.
    
    Args:
        subject (str): The subject of the email
        html_content (str): The HTML content of the email
        recipient_email (str): The email addresses of the recipients
        
    Returns:
        int: The status code of the email send operation
    """
    message = Mail(
        from_email=email_sender,
        to_emails=recipient_email,
        subject=subject,
        html_content=html_content
    )
    response = sg.send(message)
    return response.status_code

# Send emails based on gender


In [14]:
emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions_email_sender,
    tools = [send_email],
    model="gpt-4o-mini")
    
message = f"""
Send out the emails to all friends using the following information:

Male Email HTML Content:
{html_email_male.final_output}

Female Email HTML Content:
{html_email_female.final_output}

Recipients:
{recipients}

Please send the appropriate HTML email based on each recipient's gender."""

    # Get the appropriate email body based on gender
   
with trace ( "automate holidayplanner friends"):
    result= await Runner.run(emailer_agent, message)

